# 第十三讲：最大期望算法及其应用

上一讲我们介绍了无监督学习算法，现在使用的训练集都是没有类标记的数据集$\left\{x^{(1)},\cdots,x^{(m)}\right\}$。我们想要对样本进行分类并得到每个类型的样本的的密度分布。于是，我们引入潜在变量$z$，用它来表示样本所属的分布，并将数据建模为$\displaystyle P(x)=\sum_zP(x\mid z)P(z)$。在前半节课的介绍中，我们假设数据服从不同的高斯分布，并使用高斯混合模型来对样本建模，也就是令$z\sim\mathrm{Multinomial}(\phi)$，令$x\mid z=j\sim\mathcal N\left(\mu_j,\varSigma_j\right)$，使用EM算法的步骤迭代求出参数。

后半节课介绍了一般化的EM算法，该算法的目标函数为$\displaystyle\operatorname*{max}_\theta\sum_i\log p\left(x^{(i)};\theta\right)$，因为潜在变量的存在，目标函数改写为$\displaystyle\operatorname*{max}_\theta\sum_i\log\sum_z^{(i)}\log p\left(x^{(i)},z^{(i)};\theta\right)$。接下来就是使用E步骤和M步骤迭代求参数了：

* E步骤：$Q_i\left(z^{(i)}\right):=p\left(z^{(i)}\mid x^{(i)};\theta\right)$，构建目标函数$\mathscr l$的下界；
* M步骤：$\displaystyle\theta:=\mathrm{arg}\operatorname*{max}_\theta\underbrace{\sum_i\sum_{z^{(i)}}Q_i\left(z^{(i)}\right)\log\frac{p\left(x^{(i)},z^{(i)};\theta\right)}{Q_i\left(z^{(i)}\right)}}_\textrm{lower-bound function}$，最大化这个下界函数，求得新的参数。（如果把右边这个下界看做关于$\theta$的函数，那么这个下界就是$\mathscr l$的下界函数。）

再介绍一下我们为什么要使用EM算法：因为直接最大化$\mathscr l$，对参数求偏导、再将偏导置为零求参数的计算过程，因为潜在变量的存在是很求得到解析解的；而EM算法M步骤中的最大化通的式子常都是很容易求偏导并解出解析解的形式。

接着上一讲继续介绍EM算法。

怎么能确定算法一定收敛呢？设$\theta^{(t)}$和$\theta^{(t+1)}$是EM算法两次成功迭代得到的参数，只要证明$\mathscr l\left(\theta^{(t)}\right)\leq\mathscr l\left(\theta^{(t+1)}\right)$，就可以说明EM算法会随着迭代使目标函数值单调增加。而证明的关键在于$Q_i$的选择。当EM算法从参数$\theta^{(t)}$开始迭代时，我们会选择$Q_i^{(t)}\left(z^{(i)}\right):=p\left(z^{(i)}\mid x^{(i)};\theta^{(t)}\right)$，在上一讲最后，我们看到这个选择保证了延森不等式成立，并且使[上一讲](chapter12.ipynb)中的$(3)$式取等号，而因为$\displaystyle\mathscr l\left(\theta^{(t)}\right)=\sum_i\sum_{z^{(i)}}Q_i^{(t)}\left(z^{(i)}\right)\log\frac{p\left(x^{(i)},z^{(i)};\theta^{(t)}\right)}{Q_i^{(t)}\left(z^{(i)}\right)}$，则最大化等号右边的部分就可以得到参数$\theta^{(t+1)}$。因此：

$$
\begin{align}
\mathscr l\left(\theta^{(t+1)}\right)&\geq\sum_i\sum_{z^{(i)}}Q_i^{(t)}\left(z^{(i)}\right)\log\frac{p\left(x^{(i)},z^{(i)};\theta^{(t+1)}\right)}{Q_i^{(t)}\left(z^{(i)}\right)}\tag{4}\\
&\geq\sum_i\sum_{z^{(i)}}Q_i^{(t)}\left(z^{(i)}\right)\log\frac{p\left(x^{(i)},z^{(i)};\theta^{(t)}\right)}{Q_i^{(t)}\left(z^{(i)}\right)}\tag{5}\\
&=\mathscr l\left(\theta^{(t)}\right)\tag{6}
\end{align}
$$

* $(4)$式来自$\displaystyle\mathscr l(\theta)\geq\sum_i\sum_{z^{(i)}}Q_i\left(z^{(i)}\right)\log\frac{p\left(x^{(i)},z^{(i)};\theta\right)}{Q_i\left(z^{(i)}\right)}$，不等式对任意$Q_i$和$\theta$均成立，而对于$Q_i=Q_i^{(t)},\theta=\theta^{(t+1)}$时，则可以取到等号。
* 而要得到$(5)$式，需要利用$\theta^{(t+1)}$的定义式：$\displaystyle\mathrm{arg}\operatorname*{max}_\theta\sum_i\sum_{z^{(i)}}Q_i\left(z^{(i)}\right)\log\frac{p\left(x^{(i)},z^{(i)};\theta\right)}{Q_i\left(z^{(i)}\right)}$，正是这个式子使以$\theta^{(t+1)}$为参数的目标函数总是等于或大于以$\theta^{(t)}$为参数的目标函数。
* $(6)$式已经在前面出现过，为了在参数为$\theta^{(t)}$时，能够使延森不等式取到等号而选择的$Q_i^{(t)}$。

因此，EM算法能够使似然函数单调收敛。在EM算法的描述中，我们提到“运行直到收敛”，依照上面的推导，一个合理的收敛检测可以是：检查两次成功迭代后$\mathscr l(\theta)$的增量，如果增量小于某个阈值，我们就可以说“因为EM算法增长已然很慢了，所以算法已经收敛”。

**注意：**如果我们定义$\displaystyle J(Q,\theta)=\sum_i\sum_{z^{(i)}}Q_i\left(z^{(i)}\right)\log\frac{p\left(x^{(i)},z^{(i)};\theta\right)}{Q_i\left(z^{(i)}\right)}$，那么从前面的推导中可以得出$\mathscr l(\theta)\leq J(Q,\theta)$，则EM算法可以看做是函数J上的坐标上升过程，而E步骤就求$J$关于$Q$的函数的最大值，而M步骤就是求$J$关于$\theta$的函数的最大值。

## 3. 再看高斯混合模型

有了一般化定义的EM算法，我们重新审视上一讲中的高斯混合模型，拟合模型中的参数$\phi,\mu,\varSigma$。为了简洁一些，我们只看M步骤中关于$\phi,\mu_j$的参数更新过程，将$\varSigma_j$的更新留给读着考虑。

E步骤很简单，根据上面的推导，有：

$$
w_j^{(i)}=Q_i\left(z^{(i)}=j\right)=P\left(z^{(i)}=j\mid x^{(i)};\phi,\mu,\varSigma\right)=\frac{p\left(x^{(i)}\mid z^{(i)}=j;\mu,\varSigma\right)p\left(z^{(i)}=j;\phi\right)}{\sum_{i=1}^kp\left(x^{(i)}\mid z^{(i)}=l;\mu,\varSigma\right)p\left(z^{(i)}=l;\phi\right)}
$$

这里的$Q_i\left(z^{(i)}=j\right)$表示在$Q_i$分布中$z^{(i)}$取$j$的概率。

接下来是M步骤，我们需要最大化关于参数$\phi,\mu,\varSigma$的下界函数：

$$
\begin{align}
\sum_{i=1}^m\sum_{z^{(i)}}Q_i\left(z^{(i)}\right)\log\frac{p\left(x^{(i)},z^{(i)};\phi,\mu,\varSigma\right)}{Q_i\left(z^{(i)}\right)}&=\sum_{i=1}^m\sum_{j=1}^kQ_i\left(z^{z^{(i)}}=j\right)\log\frac{p\left(x^{(i)}\mid z^{(i)}=j;\mu,\varSigma\right)p\left(z^{(i)}=j;\phi\right)}{Q_i\left(z^{(i)}=j\right)}\\
&=\sum_{i=1}^m\sum_{j=1}^kw_j^{(i)}\log\frac{\frac{1}{\sqrt{(2\pi)^n\left\lvert\varSigma_j\right\rvert}}\exp\left(-\frac{1}{2}\left(x^{(i)}-\mu_j\right)^T\varSigma_j^{-1}\left(x^{(i)}-\mu_j\right)\right)\cdot\phi_j}{w_j^{(i)}}
\end{align}
$$

首先以$\mu_l$为变量做最大化，如果我们对$\mu_l$求偏导，则有：

$$
\begin{align}
\nabla_{\mu_l}\sum_{i=1}^m\sum_{j=1}^kw_j^{(i)}\log\frac{\frac{1}{\sqrt{(2\pi)^n\left\lvert\varSigma_j\right\rvert}}\exp\left(-\frac{1}{2}\left(x^{(i)}-\mu_j\right)^T\varSigma_j^{-1}\left(x^{(i)}-\mu_j\right)\right)\cdot\phi_j}{w_j^{(i)}}&=-\nabla_{\mu_l}\sum_{i=1}^m\sum_{j=1}^kw_j^{(i)}\frac{1}{2}\left(x^{(i)}-\mu_j\right)^T\varSigma_j^{-1}\left(x^{(i)}-\mu_j\right)\\
&=\frac{1}{2}\sum_{i=1}^mw_l^{(i)}\nabla_{\mu_l}2\mu_l^T\varSigma_l^{-1}x^{(i)}-\mu_l^T\varSigma_l^{-1}\mu_l\\
&=\sum_{i=1}^mw_l^{(i)}\left(\varSigma_l^{-1}x^{(i)}-\varSigma_l^{-1}\mu_l\right)
\end{align}
$$

将式子置为零再解出$\mu_l$，就得到了参数$\mu$的更新规则：

$$
\mu_l:=\frac{\sum_{i=1}^mw_l^{(i)}x^{(i)}}{\sum_{i=1}^mw^{(i)}}
$$

这就是上一讲中参数$mu$更新规则的由来。

再来推导一个M步骤中的参数$\phi$的更新规则。要对$\phi$求偏导，则将原式中含有$\phi$的项提出来，现在需要最大化：$\displaystyle\sum_{i=1}^m\sum_{j=1}^kw_j^{(i)}\log\phi_j$。但是要注意，因为$\phi_j=p\left(z^{(i)}=j;\phi\right)$是一个概率表达式，所以参数还有一个限制条件——$\phi_j$之和为$1$。为了处理限制条件$\displaystyle\sum_{j=1}^k\phi_j=1$，我们构造拉格朗日算子：

$$
\mathcal L(\phi)=\sum_{i=1}^m\sum_{j=1}^kw_j^{(i)}\log\phi_j+\beta\left(\sum_{j=1}^k\phi_j-1\right)
$$

式中的$\beta$是拉格朗日乘数（我们不需要担心$\phi_j\geq0$的限制条件，在后面会看到，从这里推导出的结论将恰好能够保证该条件成立）。求偏导得：

$$
\frac{\partial}{\partial\phi_j}\mathcal L(\phi)=\sum_{i=1}^m\frac{w_j^{(i)}}{\phi_j}+\beta
$$

将偏导置为零得到$\displaystyle\phi_j=\frac{\sum_{i=1}^mw_j^{(i)}}{-\beta}$，也就是说$\phi_j\propto\sum_{i=1}^mw_j^{(i)}$。由于$\displaystyle\sum_j\phi_j=1=\sum_{j=1}^k\frac{\sum_{i=1}^mw_j^{(i)}}{-\beta}=\frac{\sum_{i=1}^m\sum_{j=1}^kw_j^{(i)}}{-\beta}=\frac{m}{-\beta}\implies-\beta=m$（因为$w_j^{(i)}=Q_i\left(z^{(i)}=j\right)$是一个概率密度，所以有$\sum_jw_j^{(i)}=1$）。于是我们就得到了参数$\phi$的更新规则：

$$
\phi_j:=\frac{1}{m}\sum_{i=1}^mw_j^{(i)}
$$